In [21]:
import json
import time
import random
import pika

## Hello World RabbitMQ

Задача: реализовать простого производителя (producer), который отправляет сообщение "Hello World!" в очередь, и потребителя (consumer), который считывает и выводит сообщение.

Цель: понять базовый цикл работы с очередью, принцип работы обменников (exchange) с очередями (queue).

In [3]:
credentials = pika.PlainCredentials(username="user", password="password")
con_params = pika.ConnectionParameters(host="localhost", port=5670, virtual_host="/", credentials=credentials)

In [19]:
with pika.BlockingConnection(con_params) as conn:
    channel = conn.channel()
    channel.queue_declare(queue="hello_queue")
    channel.basic_publish(exchange="", routing_key="hello_queue", body=json.dumps("Hello world!"))

In [20]:
def callback(ch, method, properties, body):
    print(body.decode())

with pika.BlockingConnection(con_params) as conn:
    channel = conn.channel()
    channel.basic_consume(queue="hello_queue", on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

"Hello world!"
"Hello world!"


KeyboardInterrupt: 

## Приложение с несколькими продюсерами и потребителями

Задача: создать два продюсера, которые отправляют разные типы данных (например, показания температуры и давления с разных датчиков). Создать два потребителя, которые слушают свои очереди и обрабатывают эти данные.

Цель: освоить работу с несколькими источниками сообщений и их маршрутирование по специальным очередям.

In [ ]:
credentials = pika.PlainCredentials(username='user', password="password")
con_params = pika.ConnectionParameters(host='localhost', port=5670, virtual_host='/', credentials=credentials)

class SensorData:
    def __init__(self, temp, pres):
        self.temp = temp
        self.pres = pres

    def get_temp(self):
        return self.temp
    
    def get_pres(self):
        return self.pres

    def __call__(self, *args, **kwds):
        return {
            'temperature': self.temp,
            'pressure': self.pres,
        }

sensors = [SensorData(random.randint(0, 10), random.randint(1000, 2000)),SensorData(random.randint(0, 10), random.randint(1000, 2000)),SensorData(random.randint(0, 10), random.randint(1000, 2000)),SensorData(random.randint(0, 10), random.randint(1000, 2000))]

In [21]:
def callback(ch, method, properties, body):
    print(body.decode())

with pika.BlockingConnection(con_params) as con:
    channel = con.channel()

    channel.basic_consume(queue='test_queue', on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

{"temperature": 1, "pressure": 1986}
{"temperature": 1, "pressure": 1986}
{"temperature": 1, "pressure": 1986}


KeyboardInterrupt: 

In [20]:
with pika.BlockingConnection(con_params) as con:
    channel = con.channel()

    message = sensors[random.randint(0, 4)]()

    channel.queue_declare(queue='test_queue')
    channel.basic_publish(exchange='', routing_key='test_queue', body=json.dumps(message))


## Реализация системы очередей задач

Задача: создать очередь, куда добавляются задачи на выполнение (например, обработка изображений или отправка email). Потребители забирают задачи из очереди и обрабатывают их асинхронно.

Цель: изучить паттерн "work queue", где задачи распределяются между несколькими воркерами.


## Использование разных типов обменников (exchanges)

Задача: настроить и протестировать работу с типами обменников RabbitMQ: direct, fanout, topic, headers. Создать сценарии, когда сообщения маршрутизируются в разные очереди в зависимости от ключей маршрутизации (routing keys).

Цель: понять тонкости маршрутизации сообщений.

### Direct

- Настроить обменник direct и создать несколько очередей с уникальными ключами маршрутизации.
- Отправлять сообщения с разными ключами, чтобы проверить, что они попадают только в соответствующие очереди.
- Пример: данные с разных датчиков с ключами temperature и pressure для обработки разными сервисами.
- Цель: понять точечную (одноадресную) маршрутизацию сообщений по точному совпадению ключа.

In [26]:
credentials = pika.PlainCredentials(username='user', password="password")
con_params = pika.ConnectionParameters(host='localhost', port=5670, virtual_host='/', credentials=credentials)

exchange_name = 'sensor_data_exchange'

def setup():
    con = pika.BlockingConnection(con_params)
    channel = con.channel()

    channel.exchange_declare(exchange=exchange_name, exchange_type='direct')

    channel.queue_declare(queue='temperature_queue')
    channel.queue_declare(queue='pressure_queue')

    channel.queue_bind(exchange=exchange_name, queue='temperature_queue', routing_key='temperature')
    channel.queue_bind(exchange=exchange_name, queue='pressure_queue', routing_key='pressure')

    con.close()

In [31]:
def send_message(routing_key, message_body):
    con = pika.BlockingConnection(con_params)
    channel = con.channel()

    channel.basic_publish(
        exchange=exchange_name,
        routing_key=routing_key,
        body=json.dumps(message_body),
    )
    con.close()

In [41]:
def consume(queue_name):
    con = pika.BlockingConnection(con_params)
    channel = con.channel()

    def callback(ch, method, properties, body):
        print(f"Сообщение из очереди '{queue_name}': {json.loads(body)}")

    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)

    channel.start_consuming()

In [42]:
setup()

send_message('temperature', {'sensor_id': 1, 'value': 22.5})
send_message('pressure', {'sensor_id': 2, 'value': 1013})

In [45]:
consume('pressure_queue')

Сообщение из очереди 'pressure_queue': {'sensor_id': 2, 'value': 1013}
Сообщение из очереди 'pressure_queue': {'sensor_id': 2, 'value': 1013}
Сообщение из очереди 'pressure_queue': {'sensor_id': 2, 'value': 1013}
Сообщение из очереди 'pressure_queue': {'sensor_id': 2, 'value': 1013}


KeyboardInterrupt: 

### Fanout

- Создать fanout exchange и привязать к нему несколько очередей.
- Отправлять сообщения и убедиться, что каждое сообщение принимается всеми очередями, независимо от ключа.
- Пример: рассылка оповещений или логов во множество сервисов.
- Цель: освоить широковещательную маршрутизацию, когда каждое сообщение доставляется всем подписчикам.

In [50]:
credentials = pika.PlainCredentials('user', 'password')
connection_params = pika.ConnectionParameters('localhost', 5670, '/', credentials)

exchange_name = 'broadcast_exchange'  # имя fanout exchange

def setup():
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Создаем fanout exchange
    channel.exchange_declare(exchange=exchange_name, exchange_type='fanout')

    # Создаем две очереди
    channel.queue_declare(queue='queue_1')
    channel.queue_declare(queue='queue_2')

    # Привязываем очереди к обменнику (routing_key не используется)
    channel.queue_bind(exchange=exchange_name, queue='queue_1')
    channel.queue_bind(exchange=exchange_name, queue='queue_2')

    connection.close()

In [51]:
def send_message(message):
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Отправляем сообщение в fanout exchange
    channel.basic_publish(exchange=exchange_name, routing_key='', body=message)
    print(f"Отправлено сообщение: {message}")

    connection.close()

In [52]:
def consume(queue_name):
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    def callback(ch, method, properties, body):
        print(f"Получено в очереди '{queue_name}': {body.decode()}")

    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)
    print(f"Ожидание сообщений в очереди '{queue_name}'...")
    channel.start_consuming()

In [53]:
setup()

# Отправка сообщения (например, оповещения)
send_message("Внимание всем подписчикам: обновление системы!")

Отправлено сообщение: Внимание всем подписчикам: обновление системы!


In [54]:
consume('queue_1')

Ожидание сообщений в очереди 'queue_1'...
Получено в очереди 'queue_1': Внимание всем подписчикам: обновление системы!


KeyboardInterrupt: 

In [56]:
consume('queue_2')

Ожидание сообщений в очереди 'queue_2'...


KeyboardInterrupt: 

### Topic

- Настроить topic exchange и очереди с привязками на шаблоны ключей, например, sensor.*, device.#.
- Отправлять сообщения с ключами, соответствующими разным шаблонам, и проверять куда они попадут.
- Пример: маршрутизация логов по уровню и категории (error.ui, info.db), или данных по зонам и типам устройств.
- Цель: научиться гибкой маршрутизации с использованием шаблонов и подстановок в ключах.

In [57]:
credentials = pika.PlainCredentials('user', 'password')
connection_params = pika.ConnectionParameters('localhost', 5670, '/', credentials)

exchange_name = 'topic_sensor_exchange'

def setup():
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Создаем topic exchange
    channel.exchange_declare(exchange=exchange_name, exchange_type='topic')

    # Создаем очереди
    channel.queue_declare(queue='temp_queue')
    channel.queue_declare(queue='device_queue')

    # Привязка очередей с шаблонами ключей маршрутизации
    channel.queue_bind(exchange=exchange_name, queue='temp_queue', routing_key='sensor.temperature.*')
    channel.queue_bind(exchange=exchange_name, queue='device_queue', routing_key='device.#')

    connection.close()

In [58]:
def send_message(routing_key, message_body):
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    channel.basic_publish(
        exchange=exchange_name,
        routing_key=routing_key,
        body=json.dumps(message_body)
    )
    print(f"Отправлено сообщение с ключом '{routing_key}': {message_body}")

    connection.close()

In [59]:
def consume(queue_name):
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    def callback(ch, method, properties, body):
        data = json.loads(body)
        print(f"Сообщение из очереди '{queue_name}': {data}")

    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)
    print(f"Ожидание сообщений в очереди '{queue_name}'...")
    channel.start_consuming()

In [60]:
setup()

# Отправка сообщений с разными routing key
send_message('sensor.temperature.us', {'sensor_id': 1, 'value': 25.3})
send_message('sensor.pressure.eu', {'sensor_id': 2, 'value': 1015})
send_message('device.mobile.usa', {'device_id': 123, 'status': 'active'})
send_message('device.desktop', {'device_id': 456, 'status': 'idle'})

Отправлено сообщение с ключом 'sensor.temperature.us': {'sensor_id': 1, 'value': 25.3}
Отправлено сообщение с ключом 'sensor.pressure.eu': {'sensor_id': 2, 'value': 1015}
Отправлено сообщение с ключом 'device.mobile.usa': {'device_id': 123, 'status': 'active'}
Отправлено сообщение с ключом 'device.desktop': {'device_id': 456, 'status': 'idle'}


In [61]:
consume('device_queue')

Ожидание сообщений в очереди 'device_queue'...
Сообщение из очереди 'device_queue': {'device_id': 123, 'status': 'active'}
Сообщение из очереди 'device_queue': {'device_id': 456, 'status': 'idle'}


KeyboardInterrupt: 

In [62]:
consume('temp_queue')

Ожидание сообщений в очереди 'temp_queue'...
Сообщение из очереди 'temp_queue': {'sensor_id': 1, 'value': 25.3}


KeyboardInterrupt: 

### Headers

- Создать headers exchange и очередь с фильтром по набору заголовков сообщений (например, {"format":"pdf", "department":"sales"}).
- Отправлять сообщения с разными заголовками и проверять доставку.
- Пример: маршрутизация запросов или отчетов по сложным условиям, которые нельзя выразить простыми ключами.
- Цель: освоить маршрутизацию по метаданным сообщений, а не по ключам.

In [65]:
credentials = pika.PlainCredentials('user', 'password')
connection_params = pika.ConnectionParameters('localhost', 5670, '/', credentials)

exchange_name = 'headers_exchange'

def setup():
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Создаем headers exchange
    channel.exchange_declare(exchange=exchange_name, exchange_type='headers')

    # Очередь с условием X-Match = all (требуются все совпадения)
    channel.queue_declare(queue='pdf_sales_queue')
    channel.queue_bind(
        exchange=exchange_name,
        queue='pdf_sales_queue',
        arguments={
            'x-match': 'all',
            'format': 'pdf',
            'department': 'sales'
        }
    )

    # Очередь с условием X-Match = any (совпадение хотя бы по одному ключу)
    channel.queue_declare(queue='pdf_or_sales_queue')
    channel.queue_bind(
        exchange=exchange_name,
        queue='pdf_or_sales_queue',
        arguments={
            'x-match': 'any',
            'format': 'pdf',
            'department': 'sales'
        }
    )

    connection.close()

In [66]:
def send_message(headers, message_body):
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    properties = pika.BasicProperties(headers=headers)

    channel.basic_publish(
        exchange=exchange_name,
        routing_key='',  # routing_key игнорируется для headers exchange
        body=json.dumps(message_body),
        properties=properties
    )
    print(f"Отправлено сообщение с заголовками {headers}: {message_body}")
    connection.close()

In [67]:
def consume(queue_name):
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    def callback(ch, method, properties, body):
        data = json.loads(body)
        print(f"Получено в очереди '{queue_name}': {data} с заголовками {properties.headers}")

    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)
    print(f"Ожидание сообщений в очереди '{queue_name}'...")
    channel.start_consuming()

In [68]:
setup()

# Отправляем сообщения с различными заголовками
send_message({'format': 'pdf', 'department': 'sales'}, {'report': 'Отчет по продажам Q1'})
send_message({'format': 'pdf', 'department': 'marketing'}, {'report': 'Отчет по маркетингу Q1'})
send_message({'format': 'doc', 'department': 'sales'}, {'report': 'Отчет по продажам Q2'})
send_message({'format': 'xls', 'department': 'finance'}, {'report': 'Финансовый отчет'})

Отправлено сообщение с заголовками {'format': 'pdf', 'department': 'sales'}: {'report': 'Отчет по продажам Q1'}
Отправлено сообщение с заголовками {'format': 'pdf', 'department': 'marketing'}: {'report': 'Отчет по маркетингу Q1'}
Отправлено сообщение с заголовками {'format': 'doc', 'department': 'sales'}: {'report': 'Отчет по продажам Q2'}
Отправлено сообщение с заголовками {'format': 'xls', 'department': 'finance'}: {'report': 'Финансовый отчет'}


In [71]:
consume('pdf_or_sales_queue')

Ожидание сообщений в очереди 'pdf_or_sales_queue'...
Получено в очереди 'pdf_or_sales_queue': {'report': 'Отчет по продажам Q1'} с заголовками {'department': 'sales', 'format': 'pdf'}
Получено в очереди 'pdf_or_sales_queue': {'report': 'Отчет по маркетингу Q1'} с заголовками {'department': 'marketing', 'format': 'pdf'}
Получено в очереди 'pdf_or_sales_queue': {'report': 'Отчет по продажам Q2'} с заголовками {'department': 'sales', 'format': 'doc'}


KeyboardInterrupt: 

## Обеспечение надежности и устойчивости

Задача: настройка durable очередей и обменников, подтверждение получения сообщений (acknowledgements), повторная отправка сообщений при сбоях.

Цель: научиться конфигурировать устойчивую систему обмена сообщениями с гарантией доставки.

Для обеспечения надежности и устойчивости в RabbitMQ основными видами и способами являются:

    Durable (устойчивые) очереди и обменники
    Если очередь или обменник объявлены с параметром durable=True, они сохраняются на диске и восстанавливаются после перезапуска сервера RabbitMQ. Это значит, что настройки очередей и обменников не теряются и система сохраняет структуру маршрутизации.

    Persistent (устойчивые) сообщения
    Сообщения должны публиковаться с флагом delivery_mode=2 (persistent), чтобы RabbitMQ сохранял их на диск. Если сообщение не persistent, оно хранится только в памяти и исчезает при сбое или перезапуске.

    Подтверждения получения сообщений (acknowledgements)
    Потребитель (consumer) при обработке сообщения обязан отправлять подтверждение (ack). RabbitMQ удаляет сообщение из очереди только после такого подтверждения. Если подтверждение не пришло (например, consumer упал), сообщение будет повторно доставлено.

    Повторная отправка сообщений при сбоях
    Если подтверждение не получено, а соединение с потребителем разорвано, сообщению присваивается статус готовности к повторной доставке, и оно возвращается в очередь — так обеспечивается гарантия доставки.

In [39]:
credentials = pika.PlainCredentials('user', 'password')
connection_params = pika.ConnectionParameters('localhost', 5670, '/', credentials)

exchange_name = 'durable_exchange'
queue_name = 'durable_queue'
routing_key = 'task_key'

In [40]:
def setup():
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Создаем durable exchange
    channel.exchange_declare(exchange=exchange_name, exchange_type='direct', durable=True)

    # Создаем durable очередь
    channel.queue_declare(queue=queue_name, durable=True)

    # Привязываем очередь к exchange с routing key
    channel.queue_bind(exchange=exchange_name, queue=queue_name, routing_key=routing_key)

    connection.close()

In [41]:
def send_message(message_body):
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Отправляем persistent сообщение (delivery_mode=2)
    channel.basic_publish(
        exchange=exchange_name,
        routing_key=routing_key,
        body=json.dumps(message_body),
        properties=pika.BasicProperties(
            delivery_mode=2 # Помечаем сообщение как постоянное (persistent)
        )
    )

    print(f"Отправлено сообщение: {message_body}")
    connection.close()

In [42]:
def consume():
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    def callback(ch, method, properties, body):
        data = json.loads(body)
        print(f"Получено сообщение: {data}")

        # Симуляция обработки сообщения
        time.sleep(1)

        # Подтверждаем получение сообщения
        ch.basic_ack(delivery_tag=method.delivery_tag)
        print("Подтверждение отправлено")

    # Говорим RabbitMQ, что сообщения надо подтверждать вручную (auto_ack=False)
    channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=False)
    print("Ожидание сообщений...")
    channel.start_consuming()

In [43]:
setup()

# Отправляем несколько сообщений
for i in range(5):
    send_message({'task_number': i+1, 'description': f'Task {i+1}'})

Отправлено сообщение: {'task_number': 1, 'description': 'Task 1'}
Отправлено сообщение: {'task_number': 2, 'description': 'Task 2'}
Отправлено сообщение: {'task_number': 3, 'description': 'Task 3'}
Отправлено сообщение: {'task_number': 4, 'description': 'Task 4'}
Отправлено сообщение: {'task_number': 5, 'description': 'Task 5'}


In [44]:
consume()

Ожидание сообщений...
Получено сообщение: {'task_number': 1, 'description': 'Task 1'}
Подтверждение отправлено
Получено сообщение: {'task_number': 2, 'description': 'Task 2'}
Подтверждение отправлено
Получено сообщение: {'task_number': 3, 'description': 'Task 3'}
Подтверждение отправлено
Получено сообщение: {'task_number': 4, 'description': 'Task 4'}
Подтверждение отправлено
Получено сообщение: {'task_number': 5, 'description': 'Task 5'}
Подтверждение отправлено
Получено сообщение: {'task_number': 1, 'description': 'Task 1'}
Подтверждение отправлено
Получено сообщение: {'task_number': 2, 'description': 'Task 2'}
Подтверждение отправлено
Получено сообщение: {'task_number': 3, 'description': 'Task 3'}
Подтверждение отправлено
Получено сообщение: {'task_number': 4, 'description': 'Task 4'}
Подтверждение отправлено
Получено сообщение: {'task_number': 5, 'description': 'Task 5'}
Подтверждение отправлено
Получено сообщение: {'task_number': 1, 'description': 'Task 1'}
Подтверждение отправлено

KeyboardInterrupt: 

    Настройка Dead Letter Queue (DLQ)
    Для обработки сообщений, которые нельзя доставить или обработать, настраивается очередь с отложенными или "мертвыми" сообщениями.

    Другие параметры устойчивости

        Конфигурация автоподключений клиентов.

        Использование кластеров и зеркальных (кворумных) очередей для отказоустойчивости.

        Управление TTL (временем жизни сообщений) и политики переадресации.

Расширенная логика retry с RabbitMQ реализуется с помощью комбинации Dead Letter Queue (DLQ), TTL (time-to-live для сообщений), а также счётчиков повторных попыток, которые хранятся в заголовках сообщений.

Основная идея отложенных повторных попыток с лимитом

    При ошибке обработки сообщение отклоняется (basic_nack) без возврата в основную очередь.

    Сообщение попадает в очередь DLQ с TTL — задержкой.

    По истечении TTL сообщение автоматически перенаправляется обратно в основную очередь через Dead Letter Exchange.

    В заголовках сообщения хранится информация о количестве выполненных попыток.

    Когда количество повторных попыток достигло лимита, сообщение окончательно отвергается (отбрасывается или отправляется в другую DLQ для анализа).


In [ ]:
import pika
import json

credentials = pika.PlainCredentials('user', 'password')
connection_params = pika.ConnectionParameters('localhost', 5670, '/', credentials)

main_exchange = 'main_exchange'
dlx_exchange = 'dlx_exchange'

main_queue = 'main_queue'
delayed_queue = 'delayed_queue'
dlq_queue = 'dlq_queue'

routing_key = 'task_key'
dlx_routing_key = 'dlx_key'

MAX_RETRIES = 5  # максимум повторных попыток

def setup():
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    # Основной exchange
    channel.exchange_declare(exchange=main_exchange, exchange_type='direct', durable=True)
    # Dead letter exchange для DLQ и delayed
    channel.exchange_declare(exchange=dlx_exchange, exchange_type='direct', durable=True)

    # Основная очередь
    channel.queue_declare(queue=main_queue, durable=True, arguments={
        'x-dead-letter-exchange': dlx_exchange,
        'x-dead-letter-routing-key': dlx_routing_key
    })
    channel.queue_bind(exchange=main_exchange, queue=main_queue, routing_key=routing_key)

    # Отложенная очередь с TTL (задержка перед retry)
    channel.queue_declare(queue=delayed_queue, durable=True, arguments={
        'x-message-ttl': 10000,  # 10 секунд задержка
        'x-dead-letter-exchange': main_exchange,
        'x-dead-letter-routing-key': routing_key
    })
    channel.queue_bind(exchange=dlx_exchange, queue=delayed_queue, routing_key=dlx_routing_key)

    # Очередь «мертвых» сообщений (DLQ)
    channel.queue_declare(queue=dlq_queue, durable=True)
    channel.queue_bind(exchange=dlx_exchange, queue=dlq_queue, routing_key='dead')

    connection.close()

def send_message(body):
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    channel.basic_publish(
        exchange=main_exchange,
        routing_key=routing_key,
        body=json.dumps(body),
        properties=pika.BasicProperties(delivery_mode=2)
    )
    print(f"Отправлено сообщение: {body}")
    connection.close()

def consume():
    connection = pika.BlockingConnection(connection_params)
    channel = connection.channel()

    def callback(ch, method, properties, body):
        data = json.loads(body)
        headers = properties.headers or {}
        retry_count = 0
        if 'x-death' in headers:
            retry_count = headers['x-death'][0]['count']

        print(f"Получено сообщение: {data} | Попыток: {retry_count}")

        # Симуляция ошибки для теста retry
        if retry_count < MAX_RETRIES and data.get('fail', False):
            print("Ошибка обработки, отправляем сообщение на повтор...")
            # Покидаем текущее сообщение без подтверждения, отправляя в задержанную очередь
            ch.basic_nack(delivery_tag=method.delivery_tag, requeue=False)
        elif retry_count >= MAX_RETRIES:
            print("Достигнут лимит повторных попыток. Отправляем в DLQ.")
            # Отправляем в DLQ вручную, используя routing_key 'dead'
            ch.basic_reject(delivery_tag=method.delivery_tag, requeue=False)
        else:
            print("Обработка успешна, подтверждаем сообщение.")
            ch.basic_ack(delivery_tag=method.delivery_tag)

    channel.basic_consume(queue=main_queue, on_message_callback=callback, auto_ack=False)
    print("Ожидание сообщений...")
    channel.start_consuming()

if __name__ == '__main__':
    setup()
    send_message({'task': 'test1', 'fail': True})
    consume()


Как это работает:

    Сообщения публикуются в main_queue.

    Если при обработке возникает ошибка (fail=True), сообщение отклоняется (nack) и отправляется в delayed_queue — очередь с TTL 10 секунд.

    По истечении TTL сообщение возвращается в main_queue для повторной обработки.

    Количество попыток хранится в системном заголовке x-death.

    Если попыток больше лимита — сообщение окончательно переходит в dlq_queue (Dead Letter Queue) для анализа или ручной обработки.

Важные моменты

    Вы можете настроить разные очереди с разными TTL для увеличения задержки на каждой итерации (экспоненциальный backoff).

    Логику отказов и повторов можно адаптировать под ваши бизнес-требования.

    Такая схема минимизирует необходимость писать сложный код ре-траев в приложении, перекладывая это на RabbitMQ.

Итоги

    TTL + DLQ + анализ заголовка x-death — надежный способ настроить отложенные повторные попытки с ограничением по количеству.

    Каждое повторное доставление сообщения будет задержано заданное время.

    После достижения максимума попыток сообщение переходит в очередь ошибок.

## Мониторинг и администрирование через веб-консоль RabbitMQ

Задача: познакомиться с интерфейсом управления RabbitMQ, наблюдать статистику очередей, сообщений, пользователей.

Цель: освоить инструменты контроля и диагностики.

## Построение микросервисной архитектуры с использование RabbitMQ

Задача: разрабатывать несколько взаимосвязанных микросервисов, которые взаимодействуют через RabbitMQ, например, сервис приема заказов, обработчик оплаты и отправка уведомлений.

Цель: изучить практические сценарии использования RabbitMQ в распределенных системах.


## Проект «Логирование и аналитика событий»

Задача: интегрировать приложение, пишущее логи событий пользователя в очередь RabbitMQ, где отдельный сервис потребитель агрегирует и сохраняет данные для дальнейшего анализа.

Цель: практика потоковой обработки событий и передачи данных.